In [ ]:
import numpy as np
import pandas as pd

def h(x,w):
  return 1/(1 + np.exp(-1*np.dot(x,w))) 

def conf_matrix(y,pred):
  m = np.zeros(4)
  for i in range(len(y)):
    if y[i]==0 and pred[i]==0:
      m[0] += 1
    if y[i]==0 and pred[i]==1:
      m[1] += 1
    if y[i]==1 and pred[i]==0:
      m[2] += 1
    if y[i]==1 and pred[i]==1:
      m[3] += 1
  return m

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/NNFL Assignments (Aug 2021)/Assignment 1/data_q4_q5.xlsx")
#print(data.head())
y = np.array(data['diagnosis'], ndmin=1).T 
y = np.where(y=='M',1,0) # class labels
data.pop('diagnosis')
data.insert(0, "x0", pd.Series(np.ones(len(y)))) # appending ones
x = np.array(data) # feature matrix

# test-train-valid split
m = len(y)
trainp = int(np.floor(0.7*m)) # 70% training
validp = int(np.floor(0.1*m)) # 10% validation

y_train = y[0:trainp]
y_valid = y[trainp:trainp+validp]
y_test = y[trainp+validp:]

x_train = x[0:trainp,:]
x_valid = x[trainp:trainp+validp,:]
x_test = x[trainp+validp:,:]

# normalizing
qq = np.amax(np.abs(x_train), axis=0)
x_train = x_train/qq
x_valid = x_valid/qq
x_test = x_test/qq

#not normalizing output data for classification

m_train = len(y_train)
m_valid = len(y_valid)
m_test = len(y_test)

The following 18 cells have implementation of 9 models of Logistic Regression. Each model has one cell for grid search of optimal parameters followed by a cell for implementation of the model. The model accuracy, sensitivity and specificity on the test data is printed after implementation.

In [ ]:
#Grid Search for LOR + BGD
T_grid = list(np.linspace(300,600,7))
alpha_grid = list(np.linspace(1,2,50))

accuracy_vals = np.zeros((len(T_grid),len(alpha_grid))) #accuracy values for all models

for T in T_grid:
  for alpha in alpha_grid:
    w = np.zeros(np.shape(x)[1])
    for t in range(int(T)):
      for j in range(len(w)):
        w[j] = w[j] - (alpha/m_train)*np.dot(h(x_train,w)-y_train,x_train[:,j])
    pred = np.where(h(x_valid,w)>0.5, 1, 0)
    tn,fp,fn,tp = conf_matrix(y_valid, pred)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    accuracy_vals[T_grid.index(T)][alpha_grid.index(alpha)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal T value = {}\nOptimal alpha value = {}".format(T_grid[index[0]],alpha_grid[index[1]]))

maximum validation accuracy = 0.9821428571428571
index = (5, 43)
Optimal T value = 550.0
Optimal alpha value = 1.8775510204081631


In [ ]:
# LOR + BGD
w = np.zeros(np.shape(x)[1])
alpha = 1.8775510204081631
T = 550

for t in range(T):
  for j in range(len(w)):
    w[j] = w[j] - (alpha/m_train)*np.dot(h(x_train,w)-y_train,x_train[:,j])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9652173913043478
Sensitivity = 1.0
Specificity = 0.9550561797752809


In [ ]:
#Grid Search for LOR + L2-Norm + BGD (only for optimal Lambda)
Lambda_grid = list(np.linspace(0.001,0.01,50))

accuracy_vals = np.zeros(len(Lambda_grid)) #accuracy values for all models

for Lambda in Lambda_grid:
  w = np.zeros(np.shape(x)[1])
  for t in range(int(T)):
    for j in range(len(w)):
      w[j] = (1-alpha*Lambda)*w[j] - (alpha/m_train)*np.dot(h(x_train,w)-y_train,x_train[:,j])
  pred = np.where(h(x_valid,w)>0.5, 1, 0)
  tn,fp,fn,tp = conf_matrix(y_valid, pred)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  accuracy_vals[Lambda_grid.index(Lambda)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal Lambda value = {}".format(Lambda_grid[index[0]]))

maximum validation accuracy = 0.9821428571428571
index = (28,)
Optimal Lambda value = 0.0061428571428571435


In [ ]:
# LOR + L2-norm + BGD
w = np.zeros(np.shape(x)[1])
alpha = 1.8775510204081631
Lambda = 0.0061428571428571435
T = 550

for t in range(T):
  for j in range(len(w)):
    w[j] = (1-alpha*Lambda)*w[j] - (alpha/m_train)*np.dot(h(x_train,w)-y_train,x_train[:,j])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9739130434782609
Sensitivity = 0.9615384615384616
Specificity = 0.9775280898876404


In [ ]:
#Grid Search for LOR + L1-Norm + BGD (only for optimal Lambda)
Lambda_grid = list(np.linspace(0.01,0.1,50))

accuracy_vals = np.zeros(len(Lambda_grid)) #accuracy values for all models

for Lambda in Lambda_grid:
  w = np.zeros(np.shape(x)[1])
  for t in range(int(T)):
    for j in range(len(w)):
      w[j] = w[j] - (alpha/m_train)*np.dot(h(x_train,w)-y_train,x_train[:,j]) - (0.5*alpha*Lambda)*np.sign(w[j])
  pred = np.where(h(x_valid,w)>0.5, 1, 0)
  tn,fp,fn,tp = conf_matrix(y_valid, pred)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  accuracy_vals[Lambda_grid.index(Lambda)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal Lambda value = {}".format(Lambda_grid[index[0]]))

maximum validation accuracy = 0.9821428571428571
index = (1,)
Optimal Lambda value = 0.01183673469387755


In [ ]:
# LOR + L1-norm + BGD
w = np.zeros(np.shape(x)[1])
alpha = 1.8775510204081631
Lambda = 0.01183673469387755
T = 550

for t in range(T):
  for j in range(len(w)):
    w[j] = w[j] - (alpha/m_train)*np.dot(h(x_train,w)-y_train,x_train[:,j]) - (0.5*Lambda*alpha)*np.sign(w[j])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9652173913043478
Sensitivity = 0.9230769230769231
Specificity = 0.9775280898876404


In [ ]:
# Grid search for LOR + SGD
T_grid = list(np.linspace(450,750,7))
alpha_grid = list(np.linspace(0.1,1,50))

accuracy_vals = np.zeros((len(T_grid),len(alpha_grid))) # accuracy values of all models

for T in T_grid:
  for alpha in alpha_grid:
    w = np.zeros(np.shape(x)[1])
    for t in range(int(T)):
      ind = np.random.randint(m_train)
      for j in range(len(w)):
        w[j] = w[j] - alpha*(h(x_train,w)[ind] - y_train[ind])*x_train[ind,j]
    pred = np.where(h(x_valid,w)>0.5, 1, 0)
    tn,fp,fn,tp = conf_matrix(y_valid, pred)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    accuracy_vals[T_grid.index(T)][alpha_grid.index(alpha)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal T value = {}\nOptimal alpha value = {}".format(T_grid[index[0]],alpha_grid[index[1]]))

maximum validation accuracy = 0.9821428571428571
index = (0, 5)
Optimal T value = 450.0
Optimal alpha value = 0.19183673469387758


In [ ]:
# LOR + SGD
w = np.zeros(np.shape(x)[1])
alpha = 0.19183673469387758
T = 450

for t in range(T):
  ind = np.random.randint(m_train)
  for j in range(len(w)):
    w[j] = w[j] - alpha*(h(x_train,w)[ind] - y_train[ind])*x_train[ind,j]
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9652173913043478
Sensitivity = 0.8461538461538461
Specificity = 1.0


In [ ]:
# Grid search for LOR + L2-Norm + SGD (only for optimal Lambda)
Lambda_grid = list(np.linspace(0.001,0.01,50))

accuracy_vals = np.zeros(len(Lambda_grid)) # accuracy values of all models

for Lambda in Lambda_grid:
  w = np.zeros(np.shape(x)[1])
  for t in range(int(T)):
    ind = np.random.randint(m_train)
    for j in range(len(w)):
      w[j] = (1-alpha*Lambda)*w[j] - alpha*(h(x_train,w)[ind] - y_train[ind])*x_train[ind,j]
  pred = np.where(h(x_valid,w)>0.5, 1, 0)
  tn,fp,fn,tp = conf_matrix(y_valid, pred)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  accuracy_vals[Lambda_grid.index(Lambda)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal Lambda value = {}".format(Lambda_grid[index[0]]))

maximum validation accuracy = 0.9821428571428571
index = (1,)
Optimal Lambda value = 0.0011836734693877551


In [ ]:
# LOR + L2-Norm + SGD
w = np.zeros(np.shape(x)[1])
Lambda = 0.0011836734693877551
alpha = 0.19183673469387758
T = 450

for t in range(T):
  ind = np.random.randint(m_train)
  for j in range(len(w)):
    w[j] = (1-alpha*Lambda)*w[j] - alpha*(h(x_train,w)[ind] - y_train[ind])*x_train[ind,j]
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9565217391304348
Sensitivity = 0.8846153846153846
Specificity = 0.9775280898876404


In [ ]:
# Grid search for LOR + L1-Norm + SGD (only for optimal Lambda)
Lambda_grid = list(np.linspace(0.001,0.01,50))

accuracy_vals = np.zeros(len(Lambda_grid)) # accuracy values of all models

for Lambda in Lambda_grid:
  w = np.zeros(np.shape(x)[1])
  for t in range(int(T)):
    ind = np.random.randint(m_train)
    for j in range(len(w)):
      w[j] = w[j] - alpha*(h(x_train,w)[ind] - y_train[ind])*x_train[ind,j] - (0.5*alpha*Lambda)*np.sign(w[j])
  pred = np.where(h(x_valid,w)>0.5, 1, 0)
  tn,fp,fn,tp = conf_matrix(y_valid, pred)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  accuracy_vals[Lambda_grid.index(Lambda)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal Lambda value = {}".format(Lambda_grid[index[0]]))

maximum validation accuracy = 0.9821428571428571
index = (2,)
Optimal Lambda value = 0.0013673469387755102


In [ ]:
# LOR + L1-Norm + SGD
w = np.zeros(np.shape(x)[1])
Lambda = 0.0013673469387755102
alpha = 0.19183673469387758
T = 450

for t in range(T):
  ind = np.random.randint(m_train)
  for j in range(len(w)):
    w[j] = w[j] - alpha*(h(x_train,w)[ind] - y_train[ind])*x_train[ind,j] - (0.5*alpha*Lambda)*np.sign(w[j])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9565217391304348
Sensitivity = 0.8846153846153846
Specificity = 0.9775280898876404


In [ ]:
# Grid search for LOR + MBGD (time taken ~1h 45mins)
T_grid = list(np.linspace(350,650,7))
alpha_grid = list(np.linspace(0.1,1,50))
nb = 32
rng = np.random.default_rng()

accuracy_vals = np.zeros((len(T_grid),len(alpha_grid)))

for T in T_grid:
  for alpha in alpha_grid:
    w = np.zeros(np.shape(x)[1])
    for t in range(int(T)):
      ind = rng.choice(m_train, nb, replace=False)
      for j in range(len(w)):
        w[j] = w[j] - (alpha/nb)*np.sum([(h(x_train,w)[i]-y_train[i])*x_train[i,j] for i in ind])
    pred = np.where(h(x_valid,w)>0.5, 1, 0)
    tn,fp,fn,tp = conf_matrix(y_valid,pred)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    accuracy_vals[T_grid.index(T)][alpha_grid.index(alpha)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal T value = {}\nOptimal alpha value = {}".format(T_grid[index[0]],alpha_grid[index[1]]))

maximum validation accuracy = 0.9821428571428571
index = (0, 0)
Optimal T value = 350.0
Optimal alpha value = 0.1


In [ ]:
# LOR + MBGD
nb = 32 #batch size
rng = np.random.default_rng()

w = np.zeros(np.shape(x)[1])
alpha = 0.1
T = 350

for t in range(T):
  ind = rng.choice(m_train, nb, replace=False)
  for j in range(len(w)):
    w[j] = w[j] - (alpha/nb)*np.sum([(h(x_train,w)[i]-y_train[i])*x_train[i,j] for i in ind])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9565217391304348
Sensitivity = 0.8846153846153846
Specificity = 0.9775280898876404


In [ ]:
# Grid search for LOR + L2-Norm + MBGD (only for optimal Lambda) (time taken~10m)
nb = 32
rng = np.random.default_rng()

Lambda_grid = list(np.linspace(0.01,0.1,50))

accuracy_vals = np.zeros(len(Lambda_grid))

for Lambda in Lambda_grid:
  w = np.zeros(np.shape(x)[1])
  for t in range(int(T)):
    ind = rng.choice(m_train, nb, replace=False)
    for j in range(len(w)):
      w[j] = (1-alpha*Lambda)*w[j] - (alpha/nb)*np.sum([(h(x_train,w)[i]-y_train[i])*x_train[i,j] for i in ind])
  pred = np.where(h(x_valid,w)>0.5, 1, 0)
  tn,fp,fn,tp = conf_matrix(y_valid, pred)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  accuracy_vals[Lambda_grid.index(Lambda)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal Lambda value = {}".format(Lambda_grid[index[0]]))

maximum validation accuracy = 0.9821428571428571
index = (0,)
Optimal Lambda value = 0.01


In [ ]:
# LOR + L2-Norm + MBGD
nb = 32 #batch size
rng = np.random.default_rng()

w = np.zeros(np.shape(x)[1])
Lambda = 0.01
alpha = 0.1
T = 350

for t in range(T):
  ind = rng.choice(m_train, nb, replace=False)
  for j in range(len(w)):
    w[j] = (1-alpha*Lambda)*w[j] - (alpha/nb)*np.sum([(h(x_train,w)[i]-y_train[i])*x_train[i,j] for i in ind])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9565217391304348
Sensitivity = 0.8846153846153846
Specificity = 0.9775280898876404


In [ ]:
# Grid search for LOR + L1-Norm + MBGD (only for optimal Lambda) (time taken~10m)
nb = 32
rng = np.random.default_rng()

Lambda_grid = list(np.linspace(0.01,0.1,50))

accuracy_vals = np.zeros(len(Lambda_grid))

for Lambda in Lambda_grid:
  w = np.zeros(np.shape(x)[1])
  for t in range(int(T)):
    ind = rng.choice(m_train, nb, replace=False)
    for j in range(len(w)):
      w[j] = w[j] - (alpha/nb)*np.sum([(h(x_train,w)[i]-y_train[i])*x_train[i,j] for i in ind]) - (0.5*Lambda*alpha)*np.sign(w[j])
  pred = np.where(h(x_valid,w)>0.5, 1, 0)
  tn,fp,fn,tp = conf_matrix(y_valid, pred)
  accuracy = (tp+tn)/(tp+tn+fp+fn)
  accuracy_vals[Lambda_grid.index(Lambda)] = accuracy

# index of maximum accuracy
index = np.unravel_index(np.argmax(accuracy_vals, axis=None), accuracy_vals.shape)

print("maximum validation accuracy = {}".format(accuracy_vals[index]))
print("index = {}".format(index))
print("Optimal Lambda value = {}".format(Lambda_grid[index[0]]))

maximum validation accuracy = 0.9821428571428571
index = (0,)
Optimal Lambda value = 0.01


In [ ]:
# LOR + L1-Norm + MBGD
nb = 32 #batch size
rng = np.random.default_rng()

w = np.zeros(np.shape(x)[1])
alpha = 0.1
Lambda = 0.01
T = 350

for t in range(T):
  ind = rng.choice(m_train, nb, replace=False)
  for j in range(len(w)):
    w[j] = w[j] - (alpha/nb)*np.sum([(h(x_train,w)[i]-y_train[i])*x_train[i,j] for i in ind]) - (0.5*alpha*Lambda)*np.sign(w[j])
# training complete

# prediction and performance
pred = np.where(h(x_test,w)>0.5, 1, 0)
tn,fp,fn,tp = conf_matrix(y_test, pred)

sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
print("Accuracy = {}".format(accuracy))
print("Sensitivity = {}".format(sensitivity))
print("Specificity = {}".format(specificity))

Accuracy = 0.9652173913043478
Sensitivity = 0.9230769230769231
Specificity = 0.9775280898876404
